# Maps Kevin's data onto reactome pathways

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob

import pylab as plt
import matplotlib
from IPython.display import display, HTML

import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
from pandas import HDFStore

import seaborn as sns
from IPython.display import display

from reactome import ensembl_to_uniprot, uniprot_to_reaction, compound_to_reaction, \
    reaction_to_metabolite_pathway, get_reaction_ids, get_reaction_df, get_reaction_entities, \
    produce_kegg_dict, get_all_pathways_formulae
    
from collections import defaultdict

from ipywidgets import FloatProgress

%matplotlib inline

## Load data

In [3]:
basedir = '/Users/joewandy/Dropbox/Analysis/omics_integration'

In [4]:
time = 7
parasite = 'INFEC'
treatment = 'Unsorted'

filename = os.path.join(basedir, 'data/data_%d_%s_%s.h5' % (time, parasite, treatment))
hdf = HDFStore(filename, complevel=9, complib='bzip2')

pvt = hdf['peak_vs_transcript'].replace([np.inf, -np.inf], np.nan).fillna(0)
# pvp = hdf['peak_vs_peak']
# tvt = hdf['transcript_vs_transcript']

hdf.close()

In [5]:
pvt.columns

Index([u'ENSMUSG00000000001', u'ENSMUSG00000000028', u'ENSMUSG00000000031',
       u'ENSMUSG00000000049', u'ENSMUSG00000000056', u'ENSMUSG00000000058',
       u'ENSMUSG00000000078', u'ENSMUSG00000000085', u'ENSMUSG00000000088',
       u'ENSMUSG00000000093',
       ...
       u'ENSMUSG00000110391', u'ENSMUSG00000110393', u'ENSMUSG00000110397',
       u'ENSMUSG00000110399', u'ENSMUSG00000110404', u'ENSMUSG00000110405',
       u'ENSMUSG00000110410', u'ENSMUSG00000110414', u'ENSMUSG00000110419',
       u'ENSMUSG00000110424'],
      dtype='object', length=21794)

In [6]:
species = 'Mus musculus'

## Maps ENSEMBL IDs to UniProt IDs

In [7]:
ens_id = pvt.columns[0]
print ens_id

ENSMUSG00000000001


In [8]:
print len(pvt.columns)
ens_ids = pvt.columns.values.tolist()

21794


In [9]:
transcript_mapping = ensembl_to_uniprot(ens_ids, species, show_progress_bar=True)

FloatProgress(value=0.0, max=21794.0)

In [10]:
ensembl_ids = transcript_mapping.keys()[0:10]
for ensembl_id in ensembl_ids:
    print ensembl_id, transcript_mapping[ensembl_id]

ENSMUSG00000002010 [u'P70404']
ENSMUSG00000002015 [u'Q61335']
ENSMUSG00000026020 [u'Q6DFW4']
ENSMUSG00000074886 [u'O70293']
ENSMUSG00000006005 [u'F6ZDS4']
ENSMUSG00000003464 [u'Q8VCI5']
ENSMUSG00000004895 [u'Q9EQC8']
ENSMUSG00000046027 [u'Q9EPQ7']
ENSMUSG00000046756 [u'Q80X85']
ENSMUSG00000004892 [u'Q61361']


In [11]:
total_mapped = len(transcript_mapping)
total_genes = len(ens_ids)
certain = []
for ensembl_id in transcript_mapping:
    if len(transcript_mapping[ensembl_id]) == 1:
        certain.append(transcript_mapping[ensembl_id][0])

uncertain = total_mapped - len(certain)
print len(certain), uncertain, total_mapped, total_genes

5987 613 6600 21794


## Map UniProt IDs to Reactions

In [12]:
protein_mapping = uniprot_to_reaction(certain, species, show_progress_bar=True)

FloatProgress(value=0.0, max=5987.0)

In [13]:
for uniprot_id in certain[0:3]:
    print uniprot_id, protein_mapping[uniprot_id]
    print

P70404 [{'reaction_id': u'R-MMU-70967', 'reaction_name': u'isocitrate + NAD+ => alpha-ketoglutarate + CO2 + NADH + H+ [IDH3]'}]

Q61335 [{'reaction_id': u'R-MMU-351894', 'reaction_name': u'Caspase mediated cleavage of BAP31'}]

Q6DFW4 [{'reaction_id': u'R-MMU-4570467', 'reaction_name': u'SUMOylation of NOP58 with SUMO1'}, {'reaction_id': u'R-MMU-6791222', 'reaction_name': u'21S pre-rRNA is nucleolytically processed at site E (site2a) to yield 18SE pre-rRNA'}, {'reaction_id': u'R-MMU-6791227', 'reaction_name': u"47S pre-rRNA is nucleolytically processed at A' (01,A1), site A0, and site 02 (site 6)  to yield 45S pre-rRNA"}]



## Map KEGG Compound IDs to Reactions

Load a dataframe containing the KEGG IDs of identified compounds only

In [14]:
peak_df = pd.read_csv(os.path.join(basedir, 'data/my_analysis_peaks.csv'), index_col=0)

In [15]:
peak_df.shape

(109, 11)

In [16]:
peak_df.head()

,sec_id,mass,rt,polarity,c_id,formula,adduct,rc_id,compound,db,identifier
pid,,,,,,,,,,,
741583,1,147.076381,905.000626,positive,2825862,C5H10N2O3,M+H,3453657,L-Glutamine,kegg,C00064
741598,16,162.076139,891.966177,positive,2826034,C6H11NO4,M+H,3453882,L-2-Aminoadipate,kegg,C00956
741599,17,116.070596,771.412376,positive,2826041,C5H9NO2,M+H,3453891,L-Proline,kegg,C00148
741611,29,132.101919,677.261473,positive,2826103,C6H13NO2,M+H,3453976,L-Leucine,kegg,C00123
741680,98,132.101919,646.385592,positive,2826221,C6H13NO2,M+H,3454135,L-Leucine,kegg,C00123


In [17]:
compound_ids = sorted(list(set(peak_df['identifier'].values.tolist())))

In [18]:
compound_mapping = compound_to_reaction(compound_ids, species, show_progress_bar=True)

FloatProgress(value=0.0, max=70.0)

In [19]:
print compound_mapping.keys()[0:10]

[u'C00025', u'C00328', u'C00149', u'C00020', u'C00009', u'C00147', u'C00013', u'C00026', u'C00864', u'C00099']


## Map Reactions to Pathways

In [20]:
protein_reactions = list(set(get_reaction_ids(protein_mapping)))
compound_reactions = list(set(get_reaction_ids(compound_mapping)))
combined_reactions = list(set(protein_reactions + compound_reactions))

print len(protein_reactions)
print len(compound_reactions)
print len(combined_reactions)

7096
870
7227


In [21]:
print combined_reactions[0:10]

[u'R-MMU-9014672', u'R-MMU-2130336', u'R-MMU-5694018', u'R-MMU-6813740', u'R-MMU-9014678', u'R-MMU-204485', u'R-MMU-202626', u'R-MMU-1254386', u'R-MMU-5693561', u'R-MMU-8939706']


This is a map of reactions to pathways that are under the top-level biochemical ('Metabolism') pathways

In [22]:
pathway_mapping = reaction_to_metabolite_pathway(combined_reactions, species, show_progress_bar=True, leaf=True)
print len(pathway_mapping)
print pathway_mapping['R-MMU-2395768']

FloatProgress(value=0.0, max=7227.0)

1386
[{'pathway_id': u'R-MMU-975634', 'pathway_name': u'Retinoid metabolism and transport'}]


In [23]:
kegg_location = os.path.join(basedir, 'data/kegg/kegg.xml')
kegg_dict = produce_kegg_dict(kegg_location, 'id')

In [24]:
reaction_df = get_reaction_df(transcript_mapping, protein_mapping, compound_mapping, pathway_mapping, species)
reaction_df.sort_values(by=['compound_coverage', 'protein_coverage'], inplace=True, ascending=False)
print reaction_df.shape

(1386, 13)


In [25]:
reaction_df

,reaction_id,reaction_name,protein_coverage,compound_coverage,all_coverage,protein,observed_protein_count,all_protein_count,compound,observed_compound_count,all_compound_count,pathway_ids,pathway_names
270,R-MMU-70666,glutamate + L-glutamate gamma-semialdehyde <=>...,1.00,0.75,0.80,P29758 (ENSMUSG00000030934),1,1,C00025:C00026:C00077,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
344,R-MMU-507775,alanine + alpha-ketoglutarate <=> pyruvate + g...,1.00,0.75,0.80,Q8BGT5 (ENSMUSG00000031700),1,1,C00025:C00026:C00041,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
513,R-MMU-74213,Adenine + PRPP => AMP + PPi,1.00,0.75,0.80,P08030 (ENSMUSG00000006589),1,1,C00020:C00147:C00013,3,4,R-MMU-74217,Purine salvage
544,R-MMU-70524,pyruvate + glutamate <=> alanine + alpha-ketog...,1.00,0.75,0.80,Q8QZR5 (ENSMUSG00000022546),1,1,C00025:C00026:C00041,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
545,R-MMU-70523,alanine + alpha-ketoglutarate <=> pyruvate + g...,1.00,0.75,0.80,Q8QZR5 (ENSMUSG00000022546),1,1,C00025:C00026:C00041,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
954,R-MMU-70654,ornithine + alpha-ketoglutarate <=> glutamate ...,1.00,0.75,0.80,P29758 (ENSMUSG00000030934),1,1,C00025:C00026:C00077,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
1082,R-MMU-507749,pyruvate + glutamate <=> alanine + alpha-ketog...,1.00,0.75,0.80,Q8BGT5 (ENSMUSG00000031700),1,1,C00025:C00026:C00041,3,4,R-MMU-70614,Amino acid synthesis and interconversion (tran...
1360,R-MMU-70599,aspartate + glutamine + ATP <=> asparagine + g...,1.00,0.75,0.78,Q61024 (ENSMUSG00000029752),1,1,C00025:C00020:C00013:C00152:C00064:C00049,6,8,R-MMU-70614,Amino acid synthesis and interconversion (tran...
461,R-MMU-70560,carbamoyl phosphate + ornithine => citrulline ...,0.00,0.75,0.60,,0,1,C00009:C00077:C00327,3,4,R-MMU-70635,Urea cycle
704,R-MMU-893583,kynurenine + 2-oxoglutarate => 4-(2-aminopheny...,0.00,0.75,0.60,,0,1,C00025:C00328:C00026,3,4,R-MMU-71240,Tryptophan catabolism


In [26]:
reaction_df.to_csv('reaction_df.tsv', sep='\t', index=False)

### How many unique pathways are linked to reactions?

In [27]:
reaction_df['pathway_names'].values

array([u'Amino acid synthesis and interconversion (transamination)',
       u'Amino acid synthesis and interconversion (transamination)',
       u'Purine salvage', ..., u'Chondroitin sulfate biosynthesis',
       u'Dermatan sulfate biosynthesis', u'CS/DS degradation'], dtype=object)

In [28]:
reaction_df['pathway_ids'].values

array([u'R-MMU-70614', u'R-MMU-70614', u'R-MMU-74217', ...,
       u'R-MMU-2022870', u'R-MMU-2022923', u'R-MMU-2024101'], dtype=object)

In [29]:
pathway_ids_to_names = {}
for ids, names in zip(reaction_df['pathway_ids'].values, reaction_df['pathway_names'].values):
    pathway_ids = ids.split(':')
    pathway_names = names.split(':')
    for pid, pn in zip(pathway_ids, pathway_names):
        pathway_ids_to_names[pid] = pn

In [102]:
data = []
for pid, name in pathway_ids_to_names.iteritems():
    row = (pid, name.lower(), )
    data.append(row)
    
pathway_df = pd.DataFrame(data, columns=['pathway_id', 'pathway_name'])
pathway_df = pathway_df.sort_values(by='pathway_name').reset_index(drop=True)
display(pathway_df)

,pathway_id,pathway_name
0,R-MMU-73843,5-phosphoribose 1-diphosphate biosynthesis
1,R-MMU-1971475,a tetrasaccharide linker sequence is required ...
2,R-MMU-2161541,abacavir metabolism
3,R-MMU-156582,acetylation
4,R-MMU-399997,acetylcholine regulates insulin secretion
5,R-MMU-1482798,acyl chain remodeling of cl
6,R-MMU-1482883,acyl chain remodeling of dag and tag
7,R-MMU-1482788,acyl chain remodelling of pc
8,R-MMU-1482839,acyl chain remodelling of pe
9,R-MMU-1482925,acyl chain remodelling of pg


### Hypergeometric test

In [105]:
pw_f, pathway_id_to_name = get_all_pathways_formulae(species)

Missing formula for COMPOUND:C00013, retrieved H4P2O7 from kegg
Missing formula for COMPOUND:C00080, retrieved H from kegg
Missing formula for COMPOUND:C04637, retrieved C11H19O19P3R2 from kegg
Missing formula for COMPOUND:C05332, retrieved C8H11N from kegg
Missing formula for COMPOUND:C05261, retrieved C35H60N7O18P3S from kegg
Missing formula for COMPOUND:C00416, retrieved C5H7O8PR2 from kegg
Missing formula for COMPOUND:C05981, retrieved C11H20O22P4R2 from kegg
Missing formula for COMPOUND:C11557, retrieved C11H18O16P2R2 from kegg
Missing formula for COMPOUND:C01277, retrieved C11H18O16P2R2 from kegg
Missing formula for COMPOUND:C05974, retrieved C7H13NO9PR from kegg
Missing formula for COMPOUND:C05973, retrieved C6H13NO7PR from kegg


In [106]:
detected = set(peak_df[['formula']].values.flatten())
print detected, len(detected)

set(['C4H7N3O', 'C9H12N2O6', 'C6H12O6', 'C6H12O7', 'C4H6N4O3', 'C4H7NO4', 'C3H7NO3', 'C3H7NO2', 'C5H4N4O', 'C6H10O7', 'C5H5N5', 'C11H19NO9', 'C4H9NO3', 'C10H14N5O7P', 'C5H10O5', 'C5H9NO3', 'C5H9NO2', 'C10H17N3O6S', 'C5H6O4', 'C6H6O6', 'C7H7NO2', 'C4H9NO2', 'C5H9NO4', 'C7H14N2O4S', 'C12H22O11', 'C5H12N2O2', 'C9H13N3O5', 'C7H6O2', 'C5H7NO3', 'C3H6O3', 'C9H17NO5', 'C5H11NO2', 'C5H4N4O2', 'C2H8NO4P', 'C10H12N2O3', 'C2H7NO3S', 'C6H6N2O', 'C5H10N2O3', 'C8H11NO3', 'C7H15NO3', 'C9H11NO2', 'C6H13NO2', 'H3O4P', 'C6H13NO5', 'H4P2O7', 'C3H9O6P', 'C5H6O5', 'C5H6N2O2', 'C6H11NO4', 'C6H13N3O3', 'C2H5NO2', 'C4H8N2O3', 'C7H10O7', 'C5H8O5', 'C11H12N2O2', 'C4H8O3', 'C5H4N2O4', 'C4H6O5', 'C5H11NO2S', 'C8H15NO6', 'C5H5N5O']) 61


In [107]:
detected_f = [x for x in formulae if x in detected]
print formulae
print detected_f

set([u'C10H15N5O11P2', u'C10H15N5O10P2', u'C11H19O19P3R2', u'H3O4P', u'C10H16N5O14P3', u'H2O', u'C11H20O22P4R2', u'C10H16N5O13P3'])
[u'H3O4P']


In [108]:
data = []
for pathway_id in pw_f:
    
    pathway_name = pathway_id_to_name[pathway_id]
    
    formulae = pw_f[pathway_id]
    formulae_count = len(formulae)
    formulae_str = ','.join(sorted(formulae))
    
    detected_f = set([x for x in formulae if x in detected])
    detected_count = len(detected_f)
    detected_str = ','.join(sorted(detected_f))
    
    row = [pathway_id, pathway_name, formulae_str, formulae_count, detected_str, detected_count]
    data.append(row)
    
all_pathway_df = pd.DataFrame(data, columns=[
    'pathway_id', 'pathway_name', 
    'formula', 'formula_count',
    'detected', 'detected_count'])
all_pathway_df.set_index('pathway_id', drop=True, inplace=True)

Compute hypergeometric p-values

In [109]:
from scipy.stats import hypergeom

In [110]:
# M = the number of unique formula in all pathways in Reactome
M = len(set(','.join(all_pathway_df['formula'].values).split(',')))

# N = the number of unique formula in all pathways in the dataset
N = len(set(','.join(all_pathway_df['detected'].values).split(',')))

SMOOTHING = 1

data = []
for idx, row in all_pathway_df.iterrows():
        
    # k = the number of unique formula in the pathway of interest in the dataset
    k = row['detected_count']
    
    # n = the number of unique formula in the pathway of interest
    n = row['formula_count'] + SMOOTHING
        
    p_value = hypergeom.sf(k, M, n, N)
    assert p_value > 0
    new_row = [idx, p_value]
    data.append(new_row)

p_value_df = pd.DataFrame(data, columns=[
    'pathway_id', 'p_value'])
p_value_df.set_index('pathway_id', drop=True, inplace=True)

In [111]:
combined = pd.concat([all_pathway_df, p_value_df], axis=1)
combined = combined.sort_values(by='p_value', ascending=True).reset_index(drop=True)
display(combined)
combined.to_csv('pathway_df.csv', index=False, encoding='utf-8')

,pathway_name,formula,formula_count,detected,detected_count,p_value
0,Amino acid synthesis and interconversion (tran...,"C10H14N5O7P,C10H15N5O10P2,C10H15N5O11P2,C10H16...",29,"C10H14N5O7P,C2H7NO3S,C3H7NO2,C5H12N2O2,C5H6O5,...",8,0.000718
1,Pyrimidine catabolism,"C10H14N2O5,C10H15N2O8P,C10H15N5O10P2,C17H21N4O...",32,"C3H7NO2,C4H8N2O3,C4H9NO2,C5H10N2O3,C5H6N2O2,C9...",8,0.001574
2,Conjugation of benzoate with glycine,"C10H14N5O7P,C10H16N5O13P3,C21H36N7O16P3S,C28H4...",7,"C10H14N5O7P,C7H6O2,H4P2O7",3,0.003569
3,Urea cycle,"C10H14N5O7P,C10H15N5O10P2,C10H16N5O13P3,C10H18...",17,"C10H14N5O7P,C5H12N2O2,C6H13N3O3,H3O4P,H4P2O7",5,0.003694
4,Purine catabolism,"C10H12N4O4,C10H12N4O5,C10H13N4O7P,C10H13N4O8P,...",30,"C10H14N5O7P,C10H17N3O6S,C5H4N4O,C5H4N4O2,C5H5N...",7,0.004514
5,Phosphate bond hydrolysis by NUDT proteins,"C10H13N4O7P,C10H13N4O8P,C10H14N5O7P,C5H11O8P,H...",8,"C10H14N5O7P,H3O4P,H4P2O7",3,0.005993
6,Recycling of bile acids and salts,"C10H14N5O7P,C10H15N5O10P2,C10H16N5O13P3,C21H36...",13,"C10H14N5O7P,C2H7NO3S,H3O4P,H4P2O7",4,0.006024
7,Pyrophosphate hydrolysis,"H2O,H3O4P,H4P2O7,Mg",4,"H3O4P,H4P2O7",2,0.006783
8,Conjugation of phenylacetate with glutamine,"C10H14N5O7P,C10H16N5O13P3,C21H36N7O16P3S,H4P2O7",4,"C10H14N5O7P,H4P2O7",2,0.006783
9,Purine salvage,"C10H12N4O4,C10H12N4O5,C10H13N4O7P,C10H13N4O8P,...",27,"C10H14N5O7P,C5H4N4O,C5H5N5,C5H5N5O,H3O4P,H4P2O7",6,0.009956
